## Predicting Market Outcomes: Utilizes Data Research and DataFrames to Cluster Data

Utilized Traits:
* Betting Data (Investment Amounts, Frequency, Wallets)
* Outcome Data (Distribution, Winner)
* Market Details (Category, Date, Time Length)

Open DataFrames

In [99]:
import pandas as pd

marketOutcomes = pd.read_csv('data/silver/marketOutcomes.csv')
markets = pd.read_csv('data/silver/markets_with_ai_categories.csv')

Obtain Addresses

In [100]:
addresses = markets['marketMakerAddress']

In [101]:
import os
from dotenv import load_dotenv
import requests
import pprint
import pandas as pd
load_dotenv()
from datetime import datetime, timedelta
POLYGON_API_KEY = os.getenv("POLYGONSCAN_API_KEY")
print(POLYGON_API_KEY)
def time_decoder(address: str, iter: int = None, sz: str = None, hundredPlusBuys = None, lessThanHundredBuys = None): 
    dictForAddys = {}
    if sz is not None:
        if sz == 'large':
            dictForAddys = hundredPlusBuys[iter]
        elif sz == 'small':
            dictForAddys = lessThanHundredBuys[iter]
    
    else:
        try:
            dictForAddys = pd.read_csv(f'data/bronze/contract_buy_{address}.csv')
        except FileNotFoundError:
            return None
    try:
        
        people = [addy for addy in dictForAddys['buyer']]  
        people = [a.lower() for a in people]
        
    except KeyError:
        return None
    url = (
      f'https://api.polygonscan.com/api'
      f'?module=account'
      f'&action=tokentx'
      f'&address={address}'
      f'&startblock=0'
      f'&endblock=99999999'
      f'&apikey={POLYGON_API_KEY}'
    )
    response = requests.get(url)


    transactionDict = response.json()
    status = transactionDict.get("status")
    if status != '1':
        print(f"Error: {transactionDict.get('message')}")
        return None


    timeDict = {}
    for row in transactionDict.get('result', []):
      
      if row.get('from', '').lower() in people:
            try:
               timeDict[row.get('from','').lower()].append(row.get('timeStamp'))
            except KeyError:
               timeDict[row.get('from', '').lower()] = [row.get('timeStamp')]

    from datetime import datetime
    allTimes = [ts for ts_list in timeDict.values() for ts in ts_list]

    
    minTimestamp = min(datetime.fromtimestamp(int(x)) for x in allTimes)
    for buyer, lst in timeDict.items():
        lst_dt = sorted(datetime.fromtimestamp(int(x)) for x in lst)
        timeDict[buyer] = [
            int((ts - minTimestamp).total_seconds())   
            for ts in lst_dt
        ]
    
    buyers= dictForAddys['buyer'].str.lower()
    dictForAddys['timeStampSinceFirst'] = [timeDict[addy].pop(0) if addy in timeDict and timeDict[addy] else None for addy in buyers]
    dictForAddys.drop('timeStamp', axis=1, inplace=True)
    dictForAddys.to_csv(f'data/bronze/contract_official_buys_{address}.csv')

    return timeDict

JCBJI6AJS9E71ZVH58E8IT9E8JZBCCRQEE


In [ ]:
timerStamper = []
turn = 0
for index, market in markets.iterrows():
    turn += 1
    print(f"Turn {turn} of {len(markets)}")
    try:
        timeStamp  = time_decoder(market["marketMakerAddress"])
    except:
        continue    
    print("Yahoo News")
    try:
        buyScans = pd.read_csv(f'data/bronze/contract_official_buys_{market["marketMakerAddress"]}.csv')
    except FileNotFoundError:
        print(f"Error: 1")
        continue
    try:
        buyScans75Index = max(buyScans['timeStampSinceFirst'].tolist())*.75
        buyScans75 = buyScans[buyScans['timeStampSinceFirst'] < buyScans75Index]
        
    except:
        print(f"Error: Unicorn")
        continue
    try:
        filtered = marketOutcomes.loc[marketOutcomes["marketMakerAddress"] == market["marketMakerAddress"], "index"]
        marketOutcomeIndex = filtered.iloc[0]
    except:
        print(f"Error: 2")
        continue
    
    try:
        buysIndex0 = buyScans75[buyScans75['outcomeIndex'] == 0]['investmentAmount']
        buysIndex1 = buyScans75[buyScans75['outcomeIndex'] == 1]['investmentAmount']
    except:
        print(f"Error: 3")
        continue
    
    try:
        total = sum(buyScans75['investmentAmount'].tolist())
        buyScans75[buyScans75['investmentAmount'] > .05*total]
        buyScansVC = buyScans75['outcomeIndex'].value_counts()
        whale0 = buyScansVC.get(0, 0)
        whale1 = buyScansVC.get(1, 0)
    except:
        print("Error: 5")
        continue
    
    try: 
        sz = len(buyScans75)
        finalRatioVC = buyScans75['outcomeIndex'].value_counts()
        count0 = finalRatioVC.get(0, 0)
        count1 = finalRatioVC.get(1, 0)
        ratio = count0 / (count0 + count1)
    except:
        print("Error: 6")
        continue
        
    try:
        timerStamper.append({'Market': market['marketMakerAddress'], 'Category': market['category'], 'TimeStamps': timeStamp, 'BuysIndex0': buysIndex0.tolist(), 'BuysIndex1': buysIndex1.tolist(), 'OutcomeIndex': marketOutcomeIndex, 'Whale0': whale0, 'Whale1': whale1, 'FinalRatio': ratio})
        print("Added")
    except:
        print("Timer Failure")
        
    print("Success")
timerStamper = pd.DataFrame(timerStamper)
timerStamper.to_csv('data/silver/timerStamper1.csv', index=False)

Turn 1 of 1138
Yahoo News
Added
Success
Turn 2 of 1138
Yahoo News
Added
Success
Turn 3 of 1138
Yahoo News
Added
Success
Turn 4 of 1138
Yahoo News
Added
Success
Turn 5 of 1138
Yahoo News
Added
Success
Turn 6 of 1138
Yahoo News
Added
Success
Turn 7 of 1138
Yahoo News
Added
Success
Turn 8 of 1138
Yahoo News
Added
Success
Turn 9 of 1138
Yahoo News
Added
Success
Turn 10 of 1138
Yahoo News
Added
Success
Turn 11 of 1138
Yahoo News
Added
Success
Turn 12 of 1138
Yahoo News
Added
Success
Turn 13 of 1138
Yahoo News
Added
Success
Turn 14 of 1138
Yahoo News
Added
Success
Turn 15 of 1138
Yahoo News
Added
Success
Turn 16 of 1138
Yahoo News
Added
Success
Turn 17 of 1138
Yahoo News
Added
Success
Turn 18 of 1138
Yahoo News
Added
Success
Turn 19 of 1138
Yahoo News
Added
Success
Turn 20 of 1138
Yahoo News
Added
Success
Turn 21 of 1138
Yahoo News
Added
Success
Turn 22 of 1138
Yahoo News
Added
Success
Turn 23 of 1138
Yahoo News
Added
Success
Turn 24 of 1138
Yahoo News
Added
Success
Turn 25 of 1138
Yahoo New

Creating Official ML Model Training Data

In [158]:
import pandas as pd
timerStamper = pd.read_csv("data/silver/timerStamper1.csv")

from sklearn.preprocessing import LabelEncoder
badIndeces = []
encoder = LabelEncoder()
timerStamper['CategoryEncoded'] = encoder.fit_transform(timerStamper['Category'])
from datetime import timedelta
import ast
dataSector = []
for iter, line in timerStamper.iterrows():
    buysIndex0 = ast.literal_eval(line['BuysIndex0'])
    buysIndex1 = ast.literal_eval(line['BuysIndex1'])
    try:
        timeDict = ast.literal_eval(line['TimeStamps'])
        allDeltas = []
        for deltas in timeDict.values():
            allDeltas.extend(deltas)
        maxDelta = max(allDeltas)
    except:
        badIndeces.append(iter)
        print("Error: 7")
        continue
    # Total length of time of the market
    if None in [line['CategoryEncoded'], maxDelta ,len(buysIndex0), sum(buysIndex0), len(buysIndex1), sum(buysIndex1), line['Whale0'], line['Whale1'], line['FinalRatio']] or (len(buysIndex0) + len(buysIndex1)) < 10:
        badIndeces.append(iter)
        continue
    dataSector.append([line['CategoryEncoded'], maxDelta ,len(buysIndex0), sum(buysIndex0), len(buysIndex1), sum(buysIndex1), line['Whale0'], line['Whale1'], line['FinalRatio'], line['OutcomeIndex']])
    print(line['OutcomeIndex'])
    
timerStamper.drop(badIndeces, inplace=True)
timerStamper.reset_index(drop=True, inplace=True)
timerStamper.to_csv('data/silver/timerStamper1.csv', index=False)

1
1
1
1
1
0
0
0
1
1
1
1
1
1
0
1
1
1
0
1
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
0
0
1
1
1
1
0
1
0
0
1
0
1
1
0
0
0
0
0
1
1
0
1
0
1
0
0
1
0
1
0
0
0
0
1
0
0
1
0
0
0
1
1
0
1
1
0
1
0
1
1
1
0
1
0
1
1
1
1
1
1
0
0
1
1
0
0
1
1
0
0
1
0
0
1
1
0
0
1
0
1
0
0
1
0
0
0
1
0
0
0
1
1
1
0
0
1
0
0
0
0
0
1
1
1
0
1
1
0
1
0
0
0
0
0
1
1
0
1
1
0
0
1
1
1
1
1
1
0
1
0
1
1
0
1
0
0
1
1
0
1
0
1
1
0
0
1
0
1
1
1
0
0
1
1
0
0
1
0
0
1
0
0
0
0
1
0
1
1
1
1
1
0
1
1
1
0
0
1
1
0
0
1
1
1
0
0
1
1
0
0
0
0
1
0
0
1
1
0
1
0
0
1
1
0
0
0
0
0
0
1
0
0
1
0
1
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
0
1
0
1
1
0
1
0
1
1
0
0
0
1
1
1
1
1
1
0
1
0
1
1
0
1
1
1
0
1
1
1
1
1
0
0
1
0
0
0
1
1
1
1
0
0
1
1
0
1
0
1
1
0
1
0
1
0
0
0
0
1
0
0
1
0
0
1
1
1
1
0
1
1
0
1
1
1
0
1
1
1
0
1
0
0
1
1
1
0
0
1
0
0
0
0
0
1
0
0
1
1
1
1
0
1
1
1
0
1
0
1
0
1
0
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
0
1
0
1
1
1
0
0
0
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
0
1
1
0
0
1
0
1
0
1
0
1
1
0
1
0
0
1
1
0
0
0
1
1
1
1
1
1
0
0
0
0
0
1
0
1
1
1
0
1
0
1
1
0
1
1
1
0
1
0
1
1
1
1
1
0
0
1
0
1
1
1
0
1
1
1
0
0
1
0
1
1
1
0


Splitting Data into Test and Train

In [159]:
import torch



totalData = len(dataSector)

trainCap = totalData*4 // 5

trainData = dataSector[:trainCap]

testData = dataSector[trainCap:]

trainResults = [x[9] for x in trainData]

testResults = [x[9] for x in testData]

trainData = torch.tensor(trainData, dtype=torch.float32)
testData = torch.tensor(testData, dtype=torch.float32)
trainResults = torch.tensor(trainResults, dtype=torch.float32)
testResults = torch.tensor(testResults, dtype=torch.float32)



Creating the Model

Model Architecture taken from here : https://medium.com/data-science/pytorch-tabular-binary-classification-a0368da5bb89

Key Point: Relu serves to fid complex relationships between variables
Key Point: Normalizing to keep everything within 0 to 1 scope and not put too much weight on anything
Key Point: Linear Layers Attempt to make reason out of data

In [ ]:
import torch
import torch.nn as nn
class MarketPredictor(nn.Module):
    "Initializes multi-class classification model"
    def __init__(self, input_features=9, output_features=1, hidden_units=16):
        super().__init__()
        self.layer_1 = nn.Linear(input_features, hidden_units) 
        self.layer_2 = nn.Linear(hidden_units, hidden_units)
        self.layer_out = nn.Linear(hidden_units, output_features) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(hidden_units)
        self.batchnorm2 = nn.BatchNorm1d(hidden_units)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

MarketPredictorModel = MarketPredictor(input_features=10,
                    output_features=1,
                    hidden_units=16)

lossFn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(MarketPredictorModel.parameters(), lr=0.001)



A way to Analyze Accuracy (Literally just a percent)

In [190]:
def binaryAccuracy(actualOutcomes, predProbs, threshold=0.5):
    preds = (predProbs > threshold).float()
    correct = (preds == actualOutcomes).float().sum()
    acc = correct / actualOutcomes.shape[0]
    return acc * 100

Training Data

Both training and testing

In [191]:
def train(epochs: int):
  rates = []
  maxAcc = 0
  for epoch in range(epochs):
    MarketPredictorModel.train()
    binaryPredictions = MarketPredictorModel(trainData).squeeze()
    loss = lossFn(binaryPredictions, trainResults)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    probs1 = torch.sigmoid(binaryPredictions)
    accuracy = binaryAccuracy(trainResults, probs1)
    MarketPredictorModel.eval()
    
    with torch.inference_mode():
      testPredictions = MarketPredictorModel(testData).squeeze()
      lossTest = lossFn(testPredictions, testResults)
      probs = torch.sigmoid(testPredictions)
      accTest = binaryAccuracy(testResults, probs)
      
      
      print(f"Epoch: {epoch} | Loss: {loss:.5f} | Acc: {accuracy:.2f}% | Test Loss: {lossTest:.5f} | Test Acc: {accTest:.4f}")
      maxAcc = max(maxAcc, accTest)
      rates.append(accTest)
      
  import statistics
  return maxAcc, statistics.mean([r.item() for r in rates])

Training Data to be 85% accuracte

In [192]:
maxAcc = 0
mean = 0
while mean < 90:
    maxAcc, mean = train(50)
    print(maxAcc)
    
torch.save(MarketPredictorModel.state_dict(), "MarketPredictor.pt")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

Analyzing Parameters

In [ ]:
for name, param in MarketPredictorModel.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name} | Shape: {param.shape}")
        print(param.data)
        print("-" * 40)

Layer: layer_1.weight | Shape: torch.Size([16, 10])
tensor([[ 0.2527,  0.1686, -0.0991, -0.1218, -0.3053,  0.1996,  0.0619,  0.1634,
         -0.0460,  0.3124],
        [-0.0705,  0.1654,  0.2452, -0.2136,  0.1983,  0.0359,  0.2579, -0.1194,
         -0.0147, -0.1076],
        [-0.1191, -0.2830,  0.2830, -0.1759, -0.1469, -0.2786, -0.1452,  0.0596,
         -0.1774,  0.2405],
        [ 0.1723,  0.5217, -0.2435,  0.3013,  0.0049, -0.1029, -0.1101,  0.2035,
         -0.1072,  0.2781],
        [-0.2549,  0.2232, -0.2240,  0.0298, -0.2234,  0.1414, -0.2194, -0.0969,
         -0.0346,  0.1004],
        [ 0.2247,  0.0944, -0.1451,  0.2665, -0.1330,  0.1273, -0.1324,  0.1156,
         -0.3139,  0.2272],
        [ 0.0650,  0.0864,  0.1432,  0.1696, -0.0652,  0.1493, -0.2154,  0.0521,
          0.0038, -0.1958],
        [ 0.1252,  0.2551,  0.1134, -0.0561, -0.2127, -0.3071, -0.2998,  0.0056,
          0.2460, -0.0660],
        [ 0.0704, -0.0953,  0.2101, -0.2523,  0.2834, -0.2818,  0.2948, -0.1

In [ ]:
timerStamper = pd.read_csv("data/silver/timerStamper1.csv")

correct = 0
total  =0

for market in timerStamper['Market'].tolist():
    try:
        buyScans = pd.read_csv(f'data/bronze/contract_official_buys_{market}.csv')
    except FileNotFoundError:
        print(f"Error: 1")
        continue

    buyScans75Index = max(buyScans['timeStampSinceFirst'].tolist())*.75
    buyScans75 = buyScans[buyScans['timeStampSinceFirst'] < buyScans75Index]


    buysIndex0 = buyScans75[buyScans75['outcomeIndex'] == 0]['investmentAmount']
    buysIndex1 = buyScans75[buyScans75['outcomeIndex'] == 1]['investmentAmount']

    
    prediction = sum(buysIndex0)/(sum(buysIndex0) + sum(buysIndex1))
    prediction = 0 if prediction >= 0.5 else 1
    outcomeIndex = timerStamper[timerStamper['Market'] == market]['OutcomeIndex'].tolist()[0]
    
    if prediction == outcomeIndex:
        correct += 1
    total += 1
    
    
print(f"Correct: {correct} | Total: {total} | Accuracy: {correct/total*100}%")
    
    
    

Correct: 731 | Total: 920 | Accuracy: 79.45652173913044%


In [ ]:
trainResults.mean()

tensor(0.5774)

In [ ]:
print("trainData :", trainData.shape)
print("trainResults :", trainResults.shape)

trainData : torch.Size([736, 10])
trainResults : torch.Size([736])
